# Polytope Definition

In [1]:
#| default_exp polytope

In [2]:
#| export
import torch
import numpy as np

from typing import  Union
import gc
import warnings

from fastcore.basics import patch

In [3]:
#| export
from mhar import warningss

In [4]:
#| export

class Polytope:
    
    def __init__(self, 
                A_in:Union[torch.Tensor, np.ndarray], 
                b_in:Union[torch.Tensor, np.ndarray], 
                dtype=torch.float16,
                copy:bool=False,
                requires_grad:bool=False
                 ) -> None:
        
        self.dtype = dtype
        self.device = None
        
        self._check_dtype_()
        self._check_intra_constraint_dimensions_(A_in,b_in,'Inequality')
        
        self.copy = copy
        if copy:
            warnings.warn('The object will create a copy of the tensors, so memory usage will increase')
        else:
            warnings.warn('The object will not create a copy of the tensors, so modifications will be reflected in the object')

        if requires_grad:
            warnings.warn('The tensors will accumalate the gradient of the operations')
        self.requires_grad = requires_grad
        
        self.mI = A_in.shape[0]
        self.n = A_in.shape[1]        
        self.A_in = self._process_tensor_or_array_(A_in,'A_in')
        self.b_in = self._process_tensor_or_array_(b_in,'b_in')
        gc.collect()
    
    
    def _check_intra_constraint_dimensions_(self, 
                        A:Union[torch.Tensor, np.ndarray], 
                        b:Union[torch.Tensor, np.ndarray], 
                        constraint:str=None):
    
        assert(A.shape[0] == b.shape[0]), f"{constraint} has dimension mismatch: A has shape {A.shape} and b {b.shape}."
    
    
    def _process_tensor_or_array_(self, 
                                 A:Union[torch.Tensor, np.ndarray], 
                                 restriction:str=None):
        gc.collect()
        dtype = self.dtype
        if isinstance(A, torch.Tensor): # Torch Tensor
            # A is already a PyTorch tensor
            if self.copy:
                tensor_A = A.clone()
            else:
                tensor_A = A
            if dtype is not None and A.dtype != dtype:
                tensor_A = tensor_A.to(dtype)
            return tensor_A.requires_grad_(self.requires_grad)
        elif isinstance(A, np.ndarray): # Numpy array
            # A is a NumPy array, convert it to a PyTorch tensor
            if self.copy:
                tensor_A = torch.tensor(A, dtype=dtype)
            else:
                tensor_A = torch.from_numpy(A, dtype=dtype)
            return tensor_A.requires_grad_(self.requires_grad)
        else:
            raise ValueError(f"Input {restriction} must be a NumPy array or PyTorch tensor")

    def _check_dtype_(self):
        valid_dtypes = (torch.float16, torch.float32, torch.float64)
        assert self.dtype in valid_dtypes, f'{self.dtype} is not a valid PyTorch float data type.'
        if '16' in str(self.dtype): 
            long_message = f'The dtype {self.dtype} is typically used with GPU architectures. If you are using CPU, consider using 32 or 64-bit dtypes. \
Certain operations may be casted to 32 or 64 bits to enhance numerical stability.'

            warnings.warn(long_message)

In [5]:
#| export
@patch
def send_to_device(self:Polytope, device:str=None):
    self.A_in = self.A_in.to(device)
    self.b_in = self.b_in.to(device)
    self.device = device

In [6]:
#| export
@patch
def __str__(self:Polytope):
    string = f'Numeric Precision (dtype) {self.dtype}\n'
    string = string + f'Device: {self.device}\n'
    string = string + f'A_in: {self.A_in.shape} \n'
    string = string + f'b_in: {self.b_in.shape}'
    return string

@patch
def __repr__(self:Polytope):
    return self.__str__()

In [7]:
#| export
class NFDPolytope(Polytope):
    
    def __init__(self, 
                A_in:Union[torch.Tensor, np.ndarray], 
                b_in:Union[torch.Tensor, np.ndarray], 
                A_eq:Union[torch.Tensor, np.ndarray], 
                b_eq:Union[torch.Tensor, np.ndarray], 
                dtype=torch.float32, # Default dtype for Non-Fully-Dimensioonal Polytopes is 32 bits
                copy:bool=False,
                requires_grad:bool=False
                 ) -> None:
        
        self._check_intra_constraint_dimensions_(A_in,b_in,'Inequality')
        self._check_intra_constraint_dimensions_(A_eq,b_eq,'Equality')
        self._check_inter_constraint_dimensions_(A_in, A_eq)

        super().__init__(A_in, b_in, dtype, copy, requires_grad)
        self.mE = A_eq.shape[0]
        self.A_eq = self._process_tensor_or_array_(A_eq,'A_eq')
        self.b_eq = self._process_tensor_or_array_(b_eq,'b_eq')
        self.projection_matrix = None
        gc.collect()
        
        
    def _check_inter_constraint_dimensions_(self,A_in,A_eq):
        
        assert(A_in.shape[1] == A_eq.shape[1]), f"Constraints have dimension mismatch: A_in has shape {A_in.shape} and  A_eq {A_eq.shape}."
        
        

In [8]:
#| export
@patch
def send_to_device(self:NFDPolytope, device:str):
    Polytope.send_to_device(self,device)
    self.device = None
    self.A_eq = self.A_eq.to(device)
    self.b_eq = self.b_eq.to(device)
    if self.projection_matrix is not None:
        self.projection_matrix = self.projection_matrix.to(device)
    self.device = device

    

In [9]:
#|export
@patch
def compute_projection_matrix(self:NFDPolytope, device:str, max_precision:bool=True):
    if max_precision:
        precision = torch.float64
    else:
        precision = self.dtype
        
    if ('cuda' not in device) & ('16' in str(precision)):
        warnings.warn('Float16 precision was chosen for the polytope, but the "device=cpu" option is selected. Tensors will be temporarily cast to float32 for stability evaluation. If you wish to use float16 precision, please select "device=cuda".')
        precision = torch.float32

        
    self.send_to_device(device)
    
    # Compute (A A')^(-1)
    A_eq_t = torch.transpose(self.A_eq.to(precision), 0, 1)
    A_eq_mm_A_eq_t = torch.matmul(self.A_eq.to(precision), A_eq_t.to(precision))
    #ae_inv = torch.inverse(ae_aux)
    
    # Compute (A A')^(-1)A
    la = torch.linalg.solve(A_eq_mm_A_eq_t.to(precision),self.A_eq.to(precision)).to(precision)

    # Check numerical stability of (A A')^(-1) (AA') - I
    est = torch.mm(la, A_eq_t)
    est = torch.max(torch.abs(est - torch.eye(est.shape[0], device=device)))
    print("Max non zero error for term (A A')^(-1)A: ", est)
    del est

    # Compute I - A'(A A')^(-1)A
    #la = torch.matmul(aet, ae_inv)
    projection_matrix = torch.matmul(A_eq_t.to(precision),la.to(precision)).to(self.dtype)
    projection_matrix = torch.eye(projection_matrix.shape[0], device=device).to(self.dtype) - projection_matrix

    # Free Memory
    del A_eq_mm_A_eq_t
    del A_eq_t
    #del ae_inv
    del la
    gc.collect()
    self.projection_matrix =  projection_matrix
    if self.device is None:
        self.send_to_device(device='cpu')
    else:
        self.send_to_device(device=self.device)
        

In [11]:
#| export
@patch
def __str__(self:NFDPolytope):
    string = Polytope.__str__(self)
    string = string + f'\nA_eq: {self.A_eq.shape} \nb_eq: {self.b_eq.shape}'
    if self.projection_matrix is not None:
        string = string + f'\nProjection Matrix: {self.projection_matrix.shape}'
    return string

@patch
def __repr__(self:NFDPolytope):
    return self.__str__()